In [1]:
import pandas as pd

Proszę utworzyć dataframe z 
https://covid19.who.int/WHO-COVID-19-global-data.csv  a następnie złączyć je z danymi z pliku 

https://pkgstore.datahub.io/JohnSnowLabs/country-and-continent-codes-list/country-and-continent-codes-list-csv_csv/data/b7876b7f496677669644f3d1069d3121/country-and-continent-codes-list-csv_csv.csv 

w celu uzyskania informacji o kontynencie na którym leży kraj.

- Pogrupować dane według dnia i kontynentu, aby znaleźć sumaryczną liczbę zachorowań i zgonów (wizualizacja w załączniku covid1.png)
- Obliczyć średnią  14-dniową zachorowań dla Polski oraz Niemiec (wizualizacja w załączniku covid2.png) 

In [8]:
url_covid="https://covid19.who.int/WHO-COVID-19-global-data.csv"
url_continents="https://pkgstore.datahub.io/JohnSnowLabs/country-and-continent-codes-list/country-and-continent-codes-list-csv_csv/data/b7876b7f496677669644f3d1069d3121/country-and-continent-codes-list-csv_csv.csv"

In [4]:
df_covid = pd.read_csv(url_covid)
df_covid

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0
...,...,...,...,...,...,...,...,...
161155,2021-11-08,ZW,Zimbabwe,AFRO,19,133187,0,4685
161156,2021-11-09,ZW,Zimbabwe,AFRO,18,133205,5,4690
161157,2021-11-10,ZW,Zimbabwe,AFRO,37,133242,1,4691
161158,2021-11-11,ZW,Zimbabwe,AFRO,60,133302,3,4694


In [5]:
df_covid.describe()

,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
count,161160.000000,1.611600e+05,161160.000000,161160.000000
mean,1562.350019,3.805459e+05,31.508482,8625.974746
std,9438.312177,2.185557e+06,167.055702,42626.077423
min,-32952.000000,0.000000e+00,-60.000000,0.000000
25%,0.000000,6.000000e+01,0.000000,0.000000
50%,15.000000,5.421000e+03,0.000000,84.000000
75%,394.000000,9.539925e+04,6.000000,1629.000000
max,414188.000000,4.650153e+07,8786.000000,752960.000000


In [6]:
# usuwamy błędne dane
df_covid = df_covid.query(" New_cases>=0 and New_deaths>=0 ")

In [9]:
df_continents = pd.read_csv(url_continents)
df_continents

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0
...,...,...,...,...,...,...
257,Africa,AF,"Zambia, Republic of",ZM,ZMB,894.0
258,Oceania,OC,Disputed Territory,XX,NaN,NaN
259,Asia,AS,Iraq-Saudi Arabia Neutral Zone,XE,NaN,NaN
260,Asia,AS,United Nations Neutral Zone,XD,NaN,NaN


In [11]:
len(df_continents.Two_Letter_Country_Code.unique())

254

In [14]:
df_continents[ df_continents.duplicated(subset='Two_Letter_Country_Code') ]

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
9,Asia,AS,"Azerbaijan, Republic of",AZ,AZE,31.0
17,Asia,AS,"Armenia, Republic of",AM,ARM,51.0
59,Asia,AS,"Cyprus, Republic of",CY,CYP,196.0
84,Asia,AS,Georgia,GE,GEO,268.0
117,Asia,AS,"Kazakhstan, Republic of",KZ,KAZ,398.0
172,North America,NaN,United States Minor Outlying Islands,UM,UMI,581.0
192,Asia,AS,Russian Federation,RU,RUS,643.0
235,Asia,AS,"Turkey, Republic of",TR,TUR,792.0


In [15]:
df_continents.query(" Two_Letter_Country_Code=='RU' ")

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
191,Europe,EU,Russian Federation,RU,RUS,643.0
192,Asia,AS,Russian Federation,RU,RUS,643.0


In [16]:
df_continents.drop_duplicates(subset='Two_Letter_Country_Code', inplace=True)

In [17]:
len(df_continents.Two_Letter_Country_Code.unique()) , df_continents.shape[0]

(254, 254)

In [18]:
df = pd.merge(df_covid, df_continents, how='left', left_on="Country_code", right_on="Two_Letter_Country_Code")
df

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160978,2021-11-08,ZW,Zimbabwe,AFRO,19,133187,0,4685,Africa,AF,"Zimbabwe, Republic of",ZW,ZWE,716.0
160979,2021-11-09,ZW,Zimbabwe,AFRO,18,133205,5,4690,Africa,AF,"Zimbabwe, Republic of",ZW,ZWE,716.0
160980,2021-11-10,ZW,Zimbabwe,AFRO,37,133242,1,4691,Africa,AF,"Zimbabwe, Republic of",ZW,ZWE,716.0
160981,2021-11-11,ZW,Zimbabwe,AFRO,60,133302,3,4694,Africa,AF,"Zimbabwe, Republic of",ZW,ZWE,716.0


In [19]:
df_covid.shape

(160983, 8)

In [21]:
df.query(" Country=='Kosovo[1]' ")

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
75403,2020-01-03,XK,Kosovo[1],EURO,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
75404,2020-01-04,XK,Kosovo[1],EURO,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
75405,2020-01-05,XK,Kosovo[1],EURO,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
75406,2020-01-06,XK,Kosovo[1],EURO,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
75407,2020-01-07,XK,Kosovo[1],EURO,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76078,2021-11-08,XK,Kosovo[1],EURO,19,160829,0,2968,NaN,NaN,NaN,NaN,NaN,NaN
76079,2021-11-09,XK,Kosovo[1],EURO,8,160837,0,2968,NaN,NaN,NaN,NaN,NaN,NaN
76080,2021-11-10,XK,Kosovo[1],EURO,11,160848,0,2968,NaN,NaN,NaN,NaN,NaN,NaN
76081,2021-11-11,XK,Kosovo[1],EURO,14,160862,0,2968,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_continents.query(" Two_Letter_Country_Code=='XK' ")

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number


In [23]:
df.columns

Index(['Date_reported', 'Country_code', 'Country', 'WHO_region', 'New_cases',
       'Cumulative_cases', 'New_deaths', 'Cumulative_deaths', 'Continent_Name',
       'Continent_Code', 'Country_Name', 'Two_Letter_Country_Code',
       'Three_Letter_Country_Code', 'Country_Number'],
      dtype='object')

In [24]:
dfg = df.groupby(['Date_reported','Continent_Name'])
df1 = dfg.sum()
df1

New_cases  Cumulative_cases  New_deaths  \
Date_reported Continent_Name                                            
2020-01-03    Africa                  0                 0           0   
              Asia                    0                 0           0   
              Europe                  0                 0           0   
              North America           0                 0           0   
              Oceania                 0                 0           0   
...                                 ...               ...         ...   
2021-11-12    Asia                54488          69397554        1141   
              Europe             333571          78429810        4090   
              North America       99434          56213473        1762   
              Oceania              2081            351489          20   
              South America       22619          38605470         406   

                              Cumulative_deaths  Country_Number  
Date_reported Continent_Name                                     
2020-01-03    Africa                          0         26238.0  
              Asia                            0         18768.0  
              Europe                          0         23632.0  
              North America                   0         16685.0  
              Oceania                         0         11816.0  
...                                         ...             ...  
2021-11-12    Asia                      1076459         18768.0  
              Europe                    1457824         23632.0  
              North America             1141763         16685.0  
              Oceania                      4180         11816.0  
              South America             1174040          5200.0  

[4080 rows x 5 columns]

In [25]:
df1.reset_index(inplace=True) # przeniesienie indeksów do kolumn
df1 = df1[ ["Date_reported","Continent_Name","New_cases","New_deaths"] ]
df1.columns = ["Date","Continent","Cases","Deaths"]
df1

,Date,Continent,Cases,Deaths
0,2020-01-03,Africa,0,0
1,2020-01-03,Asia,0,0
2,2020-01-03,Europe,0,0
3,2020-01-03,North America,0,0
4,2020-01-03,Oceania,0,0
...,...,...,...,...
4075,2021-11-12,Asia,54488,1141
4076,2021-11-12,Europe,333571,4090
4077,2021-11-12,North America,99434,1762
4078,2021-11-12,Oceania,2081,20


### Obliczanie średniej 14-dniowej zachorowań dla PL i DE

In [26]:
df_covid.dtypes

Date_reported        object
Country_code         object
Country              object
WHO_region           object
New_cases             int64
Cumulative_cases      int64
New_deaths            int64
Cumulative_deaths     int64
dtype: object

In [27]:
df2 = df_covid[ (df_covid.Country=="Poland") | (df_covid.Country=="Germany")  ]
df2["Date_reported"] = pd.to_datetime(df2["Date_reported"])
df2.set_index("Date_reported", inplace=True)
df2

<ipython-input-27-d46a721983e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Date_reported"] = pd.to_datetime(df2["Date_reported"])


,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
Date_reported,,,,,,,
2020-01-03,DE,Germany,EURO,0,0,0,0
2020-01-04,DE,Germany,EURO,0,0,0,0
2020-01-05,DE,Germany,EURO,0,0,0,0
2020-01-06,DE,Germany,EURO,0,0,0,0
2020-01-07,DE,Germany,EURO,0,0,0,0
...,...,...,...,...,...,...,...
2021-11-08,PL,Poland,EURO,7314,3111534,3,77760
2021-11-09,PL,Poland,EURO,13645,3125179,220,77980
2021-11-10,PL,Poland,EURO,18546,3143725,270,78250


In [28]:
df2.index

DatetimeIndex(['2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06',
               '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10',
               '2020-01-11', '2020-01-12',
               ...
               '2021-11-03', '2021-11-04', '2021-11-05', '2021-11-06',
               '2021-11-07', '2021-11-08', '2021-11-09', '2021-11-10',
               '2021-11-11', '2021-11-12'],
              dtype='datetime64[ns]', name='Date_reported', length=1360, freq=None)

In [32]:
# liczenie średnie 14D dla Polski
covid_PL = df2.query(" Country_code=='PL' ").New_cases.resample("14D").mean()
covid_PL

Date_reported
2020-01-03        0.000000
2020-01-17        0.000000
2020-01-31        0.000000
2020-02-14        0.000000
2020-02-28        2.214286
2020-03-13       72.857143
2020-03-27      296.714286
2020-04-10      354.571429
2020-04-24      326.500000
2020-05-08      357.071429
2020-05-22      353.428571
2020-06-05      429.571429
2020-06-19      291.000000
2020-07-03      286.071429
2020-07-17      402.571429
2020-07-31      661.428571
2020-08-14      723.285714
2020-08-28      581.785714
2020-09-11      694.714286
2020-09-25     1831.857143
2020-10-09     6804.285714
2020-10-23    16925.500000
2020-11-06    23806.214286
2020-11-20    17208.857143
2020-12-04    11293.357143
2020-12-18     8787.428571
2021-01-01     8534.571429
2021-01-15     5878.785714
2021-01-29     5285.214286
2021-02-12     7328.142857
2021-02-26    12583.714286
2021-03-12    21814.071429
2021-03-26    24620.428571
2021-04-09    16553.500000
2021-04-23     6223.000000
2021-05-07     3069.500000
2021-05-21    

In [33]:
# liczenie średnie 14D dla Niemiec
covid_DE = df2.query(" Country_code=='DE' ").New_cases.resample("14D").mean()
covid_DE

Date_reported
2020-01-03        0.000000
2020-01-17        0.285714
2020-01-31        0.785714
2020-02-14        0.428571
2020-02-28      110.428571
2020-03-13     2495.785714
2020-03-27     5121.000000
2020-04-10     2846.000000
2020-04-24     1288.928571
2020-05-08      761.500000
2020-05-22      429.428571
2020-06-05      357.142857
2020-06-19      533.142857
2020-07-03      359.428571
2020-07-17      540.571429
2020-07-31      866.857143
2020-08-14     1283.714286
2020-08-28     1245.000000
2020-09-11     1621.714286
2020-09-25     2291.000000
2020-10-09     5850.357143
2020-10-23    14681.000000
2020-11-06    18452.357143
2020-11-20    17919.500000
2020-12-04    21383.714286
2020-12-18    22398.285714
2021-01-01    18489.500000
2021-01-15    14302.714286
2021-01-29     9386.071429
2021-02-12     7461.000000
2021-02-26     8447.142857
2021-03-12    12873.785714
2021-03-26    15548.000000
2021-04-09    20489.857143
2021-04-23    18270.928571
2021-05-07    10920.714286
2021-05-21    

In [35]:
df_14D = pd.concat([covid_PL, covid_DE], axis=1)
df_14D.columns = ['Polska','Niemcy']
df_14D

,Polska,Niemcy
Date_reported,,
2020-01-03,0.000000,0.000000
2020-01-17,0.000000,0.285714
2020-01-31,0.000000,0.785714
2020-02-14,0.000000,0.428571
2020-02-28,2.214286,110.428571
2020-03-13,72.857143,2495.785714
2020-03-27,296.714286,5121.000000
2020-04-10,354.571429,2846.000000
2020-04-24,326.500000,1288.928571
